# Question Answering over Resources
The notebook is based on https://python.langchain.com/en/latest/modules/chains/index_examples/qa_with_sources.html

## Prepare Data

In [16]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
import numpy as np
import os
from langchain.llms import OpenAI

import getpass
from bardapi import Bard
import os

bard_token = getpass.getpass("Enter the token for Google Bard")
openai_token = getpass.getpass("Enter the token for OpenAI")
# Input OpenAI API Key Here
os.environ["OPENAI_API_KEY"] = openai_token

Enter the token for Google Bard········
Enter the token for OpenAI········


In [17]:
import json

with open('test_data.json', 'r') as f:
    data = json.load(f)
data = data['data']
num = len(data)
print(num)
# print(data[0]['paragraphs'][0]['qas'][0])
print(data[0]['paragraphs'][0]['context'])

35
The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.


In [18]:
# ==========
# QUESTION: In what country is Normandy located?
# ANSWER1: France
# ANSWER2: Normandy is located in France.
# RESPONSE: Yes
# ==========
# QUESTION: From which countries did the Norse originate?
# ANSWER1: Denmark, Iceland and Norway 
# ANSWER2: The Norse who originated the Normans came from Denmark, Iceland, and Norway.
# RESPONSE: Yes
# ==========
# QUESTION: What is France a region of?
# ANSWER1: No relevant information found 
# ANSWER2: Sorry, I don't know.
# RESPONSE: Yes
# =========
# QUESTION: What is France a region of?
# ANSWER1: No relevant information found 
# ANSWER2: France is a region of Normandy.
# RESPONSE: No
# =========

#Along with the answer, please also give a confidence score from 1 to 10 about how confident you are about your answer.

eval_text_template = '''
You are not allowed to use any outside knowledge. You can only use what is given to you. Given one question and two answers, please response whether these ANSWER1 and ANSWER2 have the same meaning under the given question. 
The only output you can give is either "Yes" or "No". Please output only one single word. No explanation needed.
If ANSWER1 says "No relevant information found", you can only say "yes" when ANSWER2 also indicates the question cannot be answered. 

QUESTION: {question}
ANSWER1: {answer}
ANSWER2: {res_text}
'''

# ask_template = '''
# Please answer the following question only based on the given context. Do NOT use any outside knowledge. 
# If you are unsure about the answer or you cannot find the information to answer your question, you must say "Sorry, I cannot answer this question"
# ==========
# QUESTION: In what country is Normandy located?
# RESPONSE: France
# ==========

# QUESTION: What is France a region of?
# RESPONSE: Sorry, I cannot answer this question
# =========
# QUESTION: {question}
# RESPONSE: 
# '''

from langchain import PromptTemplate

template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: {context}

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["context", "query"],
    template=template
)


In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from bardapi import Bard
import time
hallucination = []
acc = []

# model = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')
model = OpenAI(temperature=0, model_name="gpt-3.5-turbo",)
chain = load_qa_with_sources_chain(model, chain_type="stuff")
# chain = load_qa_with_sources_chain(model, chain_type="map_reduce")
evaluator = Bard(token=bard_token)
bard = Bard(token=bard_token)
for i in range(num):
    for j in range(len(data[0]['paragraphs'])):
        text= data[i]['paragraphs'][j]['context']
#         text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
#         texts = text_splitter.split_text(text)
#         embeddings = OpenAIEmbeddings()
#         docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))])
        for k in range(len(data[i]['paragraphs'][j]['qas'])):
            question = data[i]['paragraphs'][j]['qas'][k]['question']
            if not data[i]['paragraphs'][j]['qas'][k]['is_impossible']:
                answer = data[i]['paragraphs'][j]['qas'][k]['answers'][0]['text']
                pos = data[i]['paragraphs'][j]['qas'][k]['answers'][0]['answer_start']
            else:
                answer = "I don't know"
                pos = None
            query = question
#             docs = docsearch.similarity_search(query, n_results = 1)
#             response = chain({"input_documents": docs, "question": ask_template.format(question=query)}, return_only_outputs=True)
#             res_text = response['output_text'].split('\n')[0]
            in_text = prompt_template.format(context=text, query=question)
            res_text = model(in_text)
            
            
            
            prompt = f"I want you to work as a question answering machine over given documents. The documents you will be given is the following paragraph: {text}. Then, I want to you answer the question based on the given paragraph. The question is {query}"

            res_text2 = bard.get_answer(prompt)['content']
            print(res_text+"||||"+res_text2)
            time.sleep(1)
            is_same = evaluator.get_answer(eval_text_template.format(question=question, answer=res_text2, res_text=res_text))['content']
            if 'yes' not in is_same.lower():
                res_text = "I don't know"
            time.sleep(0.5)
            result = evaluator.get_answer(eval_text_template.format(question=question, answer=answer, res_text=res_text))['content']
            if not data[i]['paragraphs'][j]['qas'][k]['is_impossible']:
                if 'yes' in result.lower():
                    acc.append(1)
                else:
                    acc.append(0)
            else:
                if 'yes' in result.lower():
                    hallucination.append(1)
                else:
                    hallucination.append(0)
            print(f"question: {question}\nanswer:{answer} \nresponse:{res_text}\n Correctness:{result}\n\n\n")
        print(f"acc: {np.mean(acc)}, hallucination: {np.mean(hallucination)}\n\n\n")
        break
    break

France.||||Sure, I can do that. The paragraph you provided states that Normandy is a region in France. Therefore, the answer to the question "In what country is Normandy located?" is **France**.

The paragraph also states that the Normans were descended from Norse raiders and pirates from Denmark, Iceland, and Norway. However, the Normans eventually settled in Normandy, France, and their descendants merged with the Carolingian-based cultures of West Francia. As a result, Normandy is now a part of France.
question: In what country is Normandy located?
answer:France 
response:France.
 Correctness:**Yes**

Both ANSWER1 and ANSWER2 state that Normandy is located in France. ANSWER1 is shorter and more concise, while ANSWER2 provides more emphasis on the word "France". However, both answers convey the same meaning.



The Normans were in Normandy in the 10th and 11th centuries.||||Sure, I can answer that question. The paragraph you provided states that the Normans were in Normandy from the 1

question: Who gave their name to Normandy in the 1000's and 1100's
answer:I don't know 
response:The Normans gave their name to Normandy in the 10th and 11th centuries.
 Correctness:**No.** ANSWER1 does not have the same meaning as ANSWER2. ANSWER1 states that the answerer does not know who gave their name to Normandy in the 1000's and 1100's, while ANSWER2 states that the Normans gave their name to Normandy in the 10th and 11th centuries.

ANSWER1 does not provide any information about who gave their name to Normandy, while ANSWER2 provides the specific answer that the Normans gave their name to Normandy. Therefore, ANSWER1 and ANSWER2 do not have the same meaning.



I don't know.||||France is a country in Europe. It is not a region of any other country.

The paragraph you provided states that the Normans were a people who settled in Normandy, a region in France. However, this does not mean that France is a region of another country. France is a sovereign country, and it is not part 

## Bard

In [ ]:
from bardapi import Bard
import os



In [ ]:
question = "When were the Normans in Normandy?"
response = "No relevant information found"
solution = "The Normans gave their name to Normandy in the 10th and 11th centuries."



Bard(token = token).get_answer(f"Given the question:'{question}', do the following 2 statements have the same meaning? 1.{solution}, 2.{response}. Say yes if true. Otherwise, say no.")['content']



question = "When was the French version of the word Norman first recorded?"
ans1 = "No relevant information found "
ans2 = "The French version of the word Norman was first recorded in Medieval Latin in the 9th century."

res = bard.get_answer(f"Answer the following question in a **single word**. Remember, **a single word**. Based on {question}, does {ans1} mean the same thing as {ans2}?")['content']
print(res)

# Web Scrapper

In [4]:
import pandas as pd
import numpy as np
import requests
import json, collections, time, re, string, os
from datetime import datetime

import bs4
from bs4 import BeautifulSoup

In [13]:
def retrieve_url(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup
def extract_nature_articles():
    """
    Search for and parse all coronavirus-related News article from the Nature journal that were
    published in a given period
    
    args:
        start_date (str): the lower bound of the date range to filter articles,
            has the format yyyy-mm-dd
        end_date (str): the upper bound (inclusive) of the date range to filter articles,
            has the format yyyy-mm-dd
    
    kwargs:
        base_url (str): the home page url of Nature
    
    return:
        List[str] : a list of article titles that meet the search criteria, ordered by
            date and by title
    """
    result = collections.OrderedDict()
    url = "https://www.nature.com/search?q=coronavirus&journal=nature&article_type=news&date_range=2023-2023&order=date_asc&title=coronavirus"
    soup = retrieve_url(url)
    article_list = soup.find('ul', {'class':'app-article-list-row'}).find_all('li', {'class': "app-article-list-row__item"})
    for item in article_list:
        time = item.find('div', {'class': "c-card__section"}).find('time', {'class': 'c-meta__item'})['datetime']
    
        title = item.find('h3', {'class': "c-card__title"}).text.strip()
        if time in result:
            result[time].append(title)
        else:
            result[time] = [title]
    temp = []
    for titles in result.values():
        temp.extend(sorted(titles))
    return temp

In [14]:
extract_nature_articles()

['Coronavirus variant XBB.1.5 rises in the United States — is it a global threat?',
 'The next worrisome coronavirus variant could come from China — will it get detected?',
 'The next generation of coronavirus vaccines: a graphical guide',
 'NIH reinstates grant for controversial coronavirus research']

In [30]:
text1 = """
New SARS-CoV-2 variants arise and spread with great stealth, but that hasn’t stopped Africa’s genomic sleuths from spotting a host of these threats — and alerting the rest of the world. Now an analysis details how the rapid growth in Africa’s sequencing capacity has aided global SARS-CoV-2 surveillance1. It also reveals that most variants were imported into Africa more often than they were exported from the continent.

The study, published in Science, shows that “African scientists can work together to produce high-level science”, says co-author Tulio de Oliveira, a bioinformatician at Stellenbosch University in South Africa. “Before, it was almost the norm that African scientists would work with a northern partner to produce that kind of level of science.”

Homegrown talent
In March 2020, during the early days of the pandemic, fewer than 15 of the 55 countries recognized by the African Union had the necessary infrastructure to sequence genomes. Now 39 have their own sequencing facilities. And what began as a small group of researchers in Africa meeting virtually has ballooned into a consortium involving more than 400 scientists and public-health officials from across the continent and beyond working together to track the spread of SARS-CoV-2 variants and monitor changes in the variants’ genomes.

As the consortium grew, so too did the number of sequenced genomes from Africa. By mid-2021, group members had sequenced more than 10,000 SARS-CoV-2 genomes collected on the continent2. By March this year, the count had reached 100,000.

The authors note that this marks a major milestone in African genomic surveillance. By comparison, only around 3,700 whole HIV genomes from Africa have been sequenced and publicly shared, although the virus has been circulating for decades.

“This paper is incredible,” says Jeremy Kamil, a virologist at Louisiana State University Health Shreveport. “The world needs to see more collaborations like this.”

Genomes from a total of 52 African countries have been deposited in GISAID, an online repository that is the world’s largest digital database of SARS-CoV-2 sequences. Some genomes were sequenced in the countries where they were sampled, some in other African countries and others in laboratories outside Africa.

The researchers found that in-country sequencing offered a clear speed advantage. The median turnaround — the time between the collection of a specimen from an infected individual and the addition of a completed sequence to GISAID — for genomes sequenced locally was 51 days. Median turnaround was 90 days for samples sent to other African countries and 113 days for those sent to facilities elsewhere in the world (see ‘Home-ground advantage’).

These findings “jumped out at me right away”, says Kamil. Speed is crucial when it comes to fighting the virus, he says. If a variant such as Omicron is allowed to spread for even two or three extra weeks before it’s noticed, the world’s already-slow systems for updating vaccines will have even less time to catch up, Kamil adds.

The collection of 100,000 SARS-CoV-2 genomes enabled de Oliveira and his colleagues to map when and where variants were introduced into Africa and how they had spread. The Omicron variant, for example, was first detected in South Africa in November 2021. The researchers found that although the Omicron subvariant BA.1 was exported from Africa at least 55 times — primarily to Europe and North America — it was imported into Africa at least 69 times from Europe and 102 times from North America (see ‘Virus on the go’). These import events brought the variant to African countries outside of the southern part of the continent, says study co-author Eduan Wilkinson, a bioinformatician at Stellenbosch University.

Overall, the team’s analyses indicated that most SARS-CoV-2 variants were more often introduced into Africa from other parts of the world than the other way around. “The ironic part was that Africa was punished a few times from discovery of variants,” de Oliveira says. “But a great majority of the variants, including most of the introductions of Omicron, did not come from Africa.”

De Oliveira and his colleagues plan to adapt the existing sequencing infrastructure to monitor other infectious viruses and bacteria that are of concern in Africa, such as the tuberculosis bacterium, HIV and the Lassa and Ebola viruses. “We have a lot of pathogens to deal with,” de Oliveira says.
"""

text2 = """
The US National Institutes of Health (NIH) has reinstated a grant to a highly scrutinized research organization that studies bat coronaviruses — but the agency has placed several stipulations on the scope of the research and on the organization’s accounting practices.

The move caps a years-long saga that has thrust the EcoHealth Alliance, a small non-profit organization in New York City, into the political fray for its collaborations with the Wuhan Institute of Virology (WIV) in China. In April 2020, after then-US-president Donald Trump hinted that SARS-CoV-2 originated in a WIV laboratory, the NIH terminated EcoHealth’s grant. Its goal was to study how coronaviruses, such as SARS-CoV-2, jump from bats to humans. A few months later, the NIH reinstated and immediately suspended the award until certain conditions were met that, at the time, EcoHealth said were impossible to complete.

Researchers who spoke to Nature applaud the renewal, adding that this type of research is essential to avert the next pandemic. They claim that the NIH’s termination and subsequent suspension were politically motivated, and that, although long overdue, this renewal ends — for now — a drama-filled exchange between the agency and EcoHealth.

“It’s about goddam time,” says Gerald Keusch, associate director of the National Emerging Infectious Diseases Laboratory at Boston University in Massachusetts, who organized researchers to push back against EcoHealth’s grant termination in 2020. “The integrity of science requires a barrier against political interference,” he says.

The NIH “routinely considers processes and measures for strengthening [its] oversight over federal funds” and has been working with EcoHealth to strengthen its “administrative processes to meet NIH’s expectations”, says Amanda Fine, a spokesperson for the NIH in Washington DC.

Lengthy list of conditions
Although the organization will now be able to continue its bat coronavirus research for the first time since the saga began, the NIH placed an extensive list of restrictions on the four-year, US$2.9-million award. None of the researchers who spoke to Nature had ever seen a grant with so many stipulations.

Among other things, EcoHealth is specifically forbidden from performing any in-country research in China, including with the WIV, or collecting any new samples from vertebrates — such as bats. The revised grant also mandates greater scrutiny of EcoHealth’s finances and accounting practices, driven in part by a federal watchdog report, released in January, finding that EcoHealth had misreported about $90,000 in expenses. The report also faulted the NIH for improperly terminating EcoHealth’s grant.

Furthermore, EcoHealth will be forbidden from performing any work that is deemed by the NIH’s parent organization, the Department of Health and Human Services (HHS), to have the potential to enhance the virulence or transmission of a virus. This restriction stems, in part, from criticism that research done at the WIV and funded by an EcoHealth subaward qualified as ‘gain of function’ research.

Disputed research
Congressional Republicans have alleged that this research, which involved attaching spike proteins from wild bat coronaviruses to an unrelated virus to determine whether the wild pathogens could infect human airway cells, should have undergone HHS review. Anthony Fauci, then-director of the National Institute for Allergy and Infectious Diseases, has said the agency concluded that these experiments did not meet the bar to undergo such review, and noted that the WIV did not intend to enhance the viruses.

Virologists say this type of research is essential for developing vaccines and therapeutics against emerging pathogens and for understanding how likely a pathogen is to spark a pandemic. The NIH and HHS have been finalizing guidance that will probably tighten the oversight of such research in the United States.

“I don’t know if any other single grantee from NIH has been subjected to this level of oversight,” says Peter Daszak, president of the EcoHealth Alliance. Yet he is “positive and optimistic” about the grant restart, despite all the restrictions. A key priority for the newly released funds will be analysing nearly 300 partial or complete genomes of SARS-related coronaviruses from samples that the organization collected before the funding halt, he says.

Third rail of virology
These restrictions seem reasonable, in light of the enormous public attention to and scrutiny of gain-of-function research, says Lawrence Gostin, a health-law and policy specialist at Georgetown University in Washington DC. Still, Gostin says he is surprised the agency restarted its funding for EcoHealth, given that it has been the “third rail of politics” the past few years.

Angela Rasmussen, a virologist at the University of Saskatchewan in Saskatoon, Canada, says she is pleasantly surprised to see the grant renewed, but worries about the “terrible precedent” that the NIH set by “arbitrarily” terminating an award on the basis of “unfounded rumours” regarding the origins of SARS-CoV-2. She hopes that these same restrictions will not apply to other scientists doing similar work, but is encouraged by the number of research groups that are now studying coronaviruses following the COVID-19 pandemic.
"""

In [33]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from bardapi import Bard
import time
model = OpenAI(temperature=0, model_name="gpt-3.5-turbo",)
chain = load_qa_with_sources_chain(model, chain_type="stuff")
question = 'Give separate summaries for each article'
in_text = prompt_template.format(context=text1+"\n=====\n"+text2, query=question)
res_text = model(in_text)

In [34]:
res_text

'Article 1: African scientists have played a crucial role in identifying and tracking new SARS-CoV-2 variants, with the number of sequenced genomes from Africa reaching 100,000 by March 2021. The study, published in Science, shows that African scientists can produce high-level science and work together to aid global SARS-CoV-2 surveillance. The authors note that most variants were imported into Africa more often than they were exported from the continent, and that in-country sequencing offered a clear speed advantage. The consortium of more than 400 scientists and public-health officials from across the continent and beyond plans to adapt the existing sequencing infrastructure to monitor other infectious viruses and bacteria that are of concern in Africa.\n\nArticle 2: The US National Institutes of Health (NIH) has reinstated a grant to the EcoHealth Alliance, a small non-profit organization in New York City, that studies bat coronaviruses. The move follows the termination and subseque